# Demo: CodeLlama with MLC LLM

Recently, Meta unveiled [CodeLlama](https://github.com/facebookresearch/codellama), a family of large language models for code based on Llama 2 providing state-of-the-art performance among open models, infilling capabilities, support for large input contexts, and zero-shot instruction following ability for programming tasks. This notebook demonstrates MLC LLM's support for the CodeLlama family:

- **[CodeLlama](https://github.com/nlpxucan/WizardLM/tree/main/WizardLM): a coding foundation LLM**
- **[CodeLlama-Instruct](https://github.com/nlpxucan/WizardLM/tree/main/WizardCoder): an instruction-tuned LLM for coding**
- **[CodeLlama-Python](https://github.com/nlpxucan/WizardLM/tree/main/WizardMath): a Python specialized LLM**

In this respect, MLC LLM allows everyone to develop, optimize and deploy AI models natively on everyone's devices. Therefore, making possible the deployment of coding LLMs natively, acting as **a personal AI coding assistant**.

In this notebook, we walk over the steps of using MLC LLM to run these pre-compiled CodeLlama models! We have uploaded various versions of the pre-compiled and quantized CodeLlama models here: https://huggingface.co/mlc-ai.

Learn more about MLC LLM here: https://mlc.ai/mlc-llm/docs.

Click the button below to get started!

<a target="_blank" href="https://colab.research.google.com/github/mlc-ai/notebooks/blob/main/mlc-llm/models/demo_CodeLlama.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Install MLC LLM

We will start from setting up the environment. First, let us create a new Conda environment, in which we will run the rest of the notebook.

```
conda create --name mlc-llm python=3.10
conda activate mlc-llm
```

**Google Colab**

- If you are running this in a Google Colab notebook, you would not need to create a conda environment.
- However, be sure to change your runtime to GPU by going to `Runtime` > `Change runtime type` and setting the Hardware accelerator to be "GPU".

If you are using CUDA, you can run the following command to confirm that CUDA is set up correctly, and check the driver version number as well as what GPUs are currently available for use.

In [ ]:
!nvidia-smi

Next, let's download the MLC-AI and MLC-Chat nightly build packages. If you are running in a Colab environment, then you can just run the following command. Otherwise, go to https://mlc.ai/package/ and replace the command below with the one that is appropriate for your hardware and OS.

**Google Colab**: If you are using Colab, you may see the red warnings such as "You must restart the runtime in order to use newly installed versions." For our purpose, we can disregard them, the notebook will still run correctly.

In [ ]:
!pip install --pre --force-reinstall mlc-ai-nightly-cu118 mlc-chat-nightly-cu118 -f https://mlc.ai/wheels

Let's confirm we have installed the packages successfully!

In [ ]:
!python -c "import tvm; print('tvm installed properly!')"
!python -c "import mlc_chat; print('mlc_chat installed properly!')"

## Download Prebuilt Models and Library

The following commands will download all the available prebuilt libraries (e.g., `.so` files), including the precompiled CodeLlama models. This may take a while. If in **Google Colab**, you can verify that the files are being downloaded by clicking on the folder icon on the left.

Note: If you are NOT running in **Google Colab** you may need to run this line `!conda install git git-lfs` to install `git` and `git-lfs` before running the following cell.

In [ ]:
!git lfs install

In [ ]:
!mkdir -p dist/prebuilt
!git clone https://github.com/mlc-ai/binary-mlc-llm-libs.git dist/prebuilt/lib

#### CodeLlama-7b q4f16_1 prebuilt weights

In [ ]:
!cd dist/prebuilt && git clone https://huggingface.co/mlc-ai/mlc-chat-CodeLlama-7b-hf-q4f16_1

#### CodeLlama-7b-Instruct q4f16_1 prebuilt weights

In [ ]:
!cd dist/prebuilt && git clone https://huggingface.co/mlc-ai/mlc-chat-CodeLlama-7b-Instruct-hf-q4f16_1

#### CodeLlama-7b-Python q4f16_1 prebuilt weights

In [ ]:
!cd dist/prebuilt && git clone https://huggingface.co/mlc-ai/mlc-chat-CodeLlama-7b-Python-hf-q4f16_1

In [ ]:
# Restart colab
exit()

## Let's code with CodeLlama!

Let's first try a simple code completion task with the CodeLlama-Python.

In [ ]:
from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout

In [ ]:
codellama_python = ChatModule(model="CodeLlama-7b-Python-hf-q4f16_1", device="cuda")

In [ ]:
prompt = """\
# Self-attention block implementation
class SelfAttentionBlock(nn.Module):
    def __init__("""

output = codellama_python.generate(
    prompt=prompt,
    progress_callback=StreamToStdout(callback_interval=2)
)

In [ ]:
print(prompt+output)

# Self-attention block implementation
class SelfAttentionBlock(nn.Module):
    def __init__(self, dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.key = nn.Linear(dim, dim))
        self.value = nn.Linear(dim, dim))
        self.proj = nn.Linear(dim, dim))

    def forward(self, x):
        B, N, C = x.shape
        q = self.key(x[:, :, :-64])))
        k = self.key(x[:, :, :64]]))
        v = self.value(x[:, :, :]]]]))
        attn = (q @ k.transpose(-1), v))

        x = self.proj(attn[0]]))))

        return x


In [ ]:
# Restart colab to initialize a new ChatModule
exit()

The CodeLlama models support infilling based on surrounding content. Let's try it with the foundation CodeLlama.

In [ ]:
from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout

def text_infilling(prompt: str):
    prefix = prompt.split("<FILL>")[0]
    suffix = prompt.split("<FILL>")[1]
    return f"<PRE> {prefix} <SUF>{suffix} <MID>"

def print_infilling(prompt: str, output: str):
    print(prompt.replace("<FILL>", output.replace("<EOT>", "")))

In [ ]:
codellama = ChatModule(model="CodeLlama-7b-hf-q4f16_1", device="cuda")

In [ ]:
prompt = """\
# Installation instructions:
    <FILL>
This downloads the LLaMA inference code and installs the repository as a local pip package.
"""

output = codellama.generate(
    prompt=text_infilling(prompt),
    progress_callback=StreamToStdout(callback_interval=2)
)

In [ ]:
print_infilling(prompt, output)

# Installation instructions:
    1. Clone the repository.
    ```
    git clone https://github.com/LLaMA/LLaMA.git
    ```


2. Install the pip package.
    ```
    cd LLaMA
    pip install -e .
    ```
 
This downloads the LLaMA inference code and installs the repository as a local pip package.



In [ ]:
# Restart colab to create a new ChatModule
exit()

Finally, the CodeLlama-Instruct has instruction following ability for programming tasks.

In [ ]:
from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout

In [ ]:
codellama_instruct = ChatModule(model="CodeLlama-7b-Instruct-hf-q4f16_1", device="cuda")

In [ ]:
prompt = ("Write a C++ program that computes the set of sums of all contiguous"
          "sublists of a given list.")

output = codellama_instruct.generate(
    prompt=prompt,
    progress_callback=StreamToStdout(callback_interval=2)
)

Here is a C++ program that computes the set of sums of all contiguous sublists of a given list:
#include <iostream>
using namespace std;
void computeSums(const list<int> &lst, list<int> &sums) {
    // Initialize the sums list
    sums.clear();
    // Compute the sums of all contiguous sublists
    for (int i = 0; i < lst.size() - 1; i++) {
        int sum = 0;
        for (int j = i; j < lst.size() - 1; j++) {
            sum += lst[j];
        }
        sums.push_back(sum));
    }
    // Print the sums list
    for (int i = 0; i < sums.size(); i++) {
        cout << sums[i] << endl;
    }
}
int main() {
    list<int> lst = {1, 2, 3, 4, 5};
    list<int> sums;
    computeSums(lst, sums);
    return 0;
}
This program takes a list of integers as input, and computes the set of sums of all contiguous sublists of the input list. The program then prints the computed set of sums.
Note that the input list must be a list of integers, and that the program will produce an error if the input list

In [ ]:
output = codellama_instruct.generate(
    prompt="Write this in Java instead.",
    progress_callback=StreamToStdout(callback_interval=2)
)

Here is the C++ program written in Java instead:
import java.util.ArrayList;
public class SumsOfSublists {
    public static void main(String[] args) {
        ArrayList<Integer> lst = new ArrayList<Integer>(){{add(1);add(2);add(3);add(4);add(5);}};
        ArrayList<Integer> sums = new ArrayList<Integer>();
        computeSums(lst, sums));
        for (int i = 0; i < sums.size(); i++) {
            System.out.println(sums[i])));
        }
    }
    public static void computeSums(ArrayList<Integer> lst, ArrayList<Integer> sums) {
        for (int i = 0; i < lst.size() - 1; i++) {
            int sum = 0;
            for (int j = i; j < lst.size() - 1; j++) {
                sum += lst[j]);
            }
            sums.add(sum));
        }
    }
}
This Java program takes a list of integers as input, and computes the set of sums of all contiguous sublists of the input list. The program then prints the computed set of sums.
Note that the input list must be a list of integers, and that t

In [ ]:
# Restart colab to create a new ChatModule
exit()